# Lambda Function Bug
For constrint, if we use lambda functions, it will produce the data with same value. As below.

In [1]:
import numpy as np
import pandas as pd
from sdv.tabular import CopulaGAN, GaussianCopula
import sdv.constraints as cons
X=np.random.randint(0,5,10000)
Y=np.random.randint(0,5,10000)
Z=np.random.randint(0,5,10000)
df = pd.DataFrame(pd.DataFrame([X, Y, Z]).values.T,columns = ["X", "Y", "Z"])

def MAX(a,b): return (abs(a-b)+(a+b))//2
def MIN(a,b): return (-abs(a-b)+(a+b))//2
feats_funcs = {feat : (lambda data : MIN(MAX(data[feat], 0), 4))  for feat in ["X", "Y", "Z"]}
feats_constraints = [cons.ColumnFormula(column = feat,
                                        formula = feats_funcs[feat],
                                        handling_strategy = 'reject_sampling') for feat in ["X", "Y", "Z"]]

In [2]:
real_data = df
model = GaussianCopula(constraints = feats_constraints)
model.fit(real_data)
synth_data = model.sample(600, max_retries = 3)

c:\users\1\.julia\conda\3\envs\py36\lib\site-packages\scipy\stats\_continuous_distns.py:4965: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
c:\users\1\.julia\conda\3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:2429: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


In [3]:
synth_data

,X,Y,Z
59,2,2,2
70,2,2,2
78,3,3,3
82,1,1,1
104,2,2,2
...,...,...,...
664,3,3,3
671,2,2,2
717,2,2,2
741,1,1,1


Also, it does not output an error if the function has not enough data (It should output when max_retries is reached.) However, if we change to the defined function, the error disappeared.

In [4]:
def feats_valid(feat):
    def foo(data):
        return MIN(MAX(data[feat], 0), 4) 
    return foo
feats_constraints = [cons.ColumnFormula(column = feat,
                                        formula = feats_valid(feat),
                                        handling_strategy = 'reject_sampling') for feat in ["X", "Y", "Z"]]

In [5]:
real_data = df
model = GaussianCopula(constraints = feats_constraints)
model.fit(real_data)
synth_data = model.sample(600, max_retries = 3)

c:\users\1\.julia\conda\3\envs\py36\lib\site-packages\scipy\stats\_continuous_distns.py:4965: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
c:\users\1\.julia\conda\3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:2429: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


In [6]:
synth_data

,X,Y,Z
0,1,1,2
1,1,2,3
2,1,4,1
3,1,1,0
6,0,2,3
...,...,...,...
5,2,1,0
6,2,3,2
7,1,3,3
8,1,1,3
